In [19]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr, mean_squared_error as mse
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline
import torch
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt

In [29]:
folder_path = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111'  # file_list = [f for f in os.listdir(folder_path) if f.endswith('_10.png')][:20]

In [21]:
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("mps")

Loading pipeline components...: 100%|██████████| 6/6 [00:09<00:00,  1.56s/it]


In [22]:
def generate_upscaled_images(image_path, prompt="high resolution image of a histopathology slide"):
    low_res_img = Image.open(image_path).convert("RGB")
    upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
    return upscaled_image

In [23]:
def zoom_in(image_path, output_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found")
        return None

    height, width = img.shape[:2]
    new_width, new_height = width // 2, height // 2
    x1, y1 = width // 4, height // 4
    x2, y2 = x1 + new_width, y1 + new_height
    cropped_img = img[y1:y2, x1:x2]

    cv2.imwrite(output_path, cropped_img)
    return cropped_img

In [32]:
results = []
sample_images_20 = []
sample_images_40 = []

for idx, file in enumerate(file_list):
    base_name = file[:-7]
    image_10_path = os.path.join(folder_path, file)
    image_20_path = os.path.join(folder_path, f"{base_name}_20.png")
    image_40_path = os.path.join(folder_path, f"{base_name}_40.png")

    # Generate images
    upscaled_image = generate_upscaled_images(image_10_path)
    upscaled_image_path = os.path.join(folder_path, f"{base_name}_10_upsampled.png")
    upscaled_image.save(upscaled_image_path)
    
    zoomed_image_20x = zoom_in(upscaled_image_path, os.path.join(folder_path, f"{base_name}_20_generated.png"))
    zoomed_image_40x = zoom_in(os.path.join(folder_path, f"{base_name}_20_generated.png"), os.path.join(folder_path, f"{base_name}_40_generated.png"))
    
    # Load original 20x and 40x images
    image_20 = cv2.imread(image_20_path)
    image_40 = cv2.imread(image_40_path)
    
    # Load generated 20x and 40x images
    image_20_generated = cv2.imread(os.path.join(folder_path, f"{base_name}_20_generated.png"))
    image_40_generated = cv2.imread(os.path.join(folder_path, f"{base_name}_40_generated.png"))
    
    # Downsample generated images to match original image dimensions
    image_20_generated = downsample_image(image_20_generated, image_20.shape)
    image_40_generated = downsample_image(image_40_generated, image_40.shape)
    
    # Print dimensions for debugging
    print(f"{base_name}_20.png dimensions: {image_20.shape}")
    print(f"{base_name}_20_generated.png dimensions: {image_20_generated.shape}")
    print(f"{base_name}_40.png dimensions: {image_40.shape}")
    print(f"{base_name}_40_generated.png dimensions: {image_40_generated.shape}")
   

100%|██████████| 75/75 [09:17<00:00,  7.43s/it]


217_20.png dimensions: (224, 224, 3)
217_20_generated.png dimensions: (224, 224, 3)
217_40.png dimensions: (224, 224, 3)
217_40_generated.png dimensions: (224, 224, 3)


100%|██████████| 75/75 [09:27<00:00,  7.57s/it]


546_20.png dimensions: (224, 224, 3)
546_20_generated.png dimensions: (224, 224, 3)
546_40.png dimensions: (224, 224, 3)
546_40_generated.png dimensions: (224, 224, 3)


100%|██████████| 75/75 [09:45<00:00,  7.81s/it]


...

1502_20.png dimensions: (224, 224, 3)
1502_20_generated.png dimensions: (224, 224, 3)
1502_40.png dimensions: (224, 224, 3)
1502_40_generated.png dimensions: (224, 224, 3)


100%|██████████| 75/75 [09:12<00:00,  7.36s/it]


1199_20.png dimensions: (224, 224, 3)
1199_20_generated.png dimensions: (224, 224, 3)
1199_40.png dimensions: (224, 224, 3)
1199_40_generated.png dimensions: (224, 224, 3)


 15%|█▍        | 11/75 [01:46<10:20,  9.69s/it]


KeyboardInterrupt: 

In [26]:
def downsample_image(image, target_shape):
    return cv2.resize(image, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_AREA)


In [24]:
def calculate_fid_kid(image1, image2):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    image1 = cv2.resize(image1, (299, 299))
    image2 = cv2.resize(image2, (299, 299))
    image1 = preprocess_input(image1)
    image2 = preprocess_input(image2)
    image1 = np.expand_dims(image1, axis=0)
    image2 = np.expand_dims(image2, axis=0)
    
    act1 = model.predict(image1)
    act2 = model.predict(image2)
    
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    
    kid = np.mean(np.square(act1 - act2))
    
    return fid, kid

In [28]:
results = []
sample_images_20 = []
sample_images_40 = []

for idx, file in enumerate(file_list):
    base_name = file[:-7]
    image_10_path = os.path.join(folder_path, file)
    image_20_path = os.path.join(folder_path, f"{base_name}_20.png")
    image_40_path = os.path.join(folder_path, f"{base_name}_40.png")

    # Generate images
    upscaled_image = generate_upscaled_images(image_10_path)
    upscaled_image_path = os.path.join(folder_path, f"{base_name}_10_upsampled.png")
    upscaled_image.save(upscaled_image_path)
    
    zoomed_image_20x = zoom_in(upscaled_image_path, os.path.join(folder_path, f"{base_name}_20_generated.png"))
    zoomed_image_40x = zoom_in(os.path.join(folder_path, f"{base_name}_20_generated.png"), os.path.join(folder_path, f"{base_name}_40_generated.png"))
    
    # Load original 20x and 40x images
    image_20 = cv2.imread(image_20_path)
    image_40 = cv2.imread(image_40_path)
    
    # Load generated 20x and 40x images
    image_20_generated = cv2.imread(os.path.join(folder_path, f"{base_name}_20_generated.png"))
    image_40_generated = cv2.imread(os.path.join(folder_path, f"{base_name}_40_generated.png"))
    
    # Downsample generated images to match original image dimensions
    image_20_generated = downsample_image(image_20_generated, image_20.shape)
    image_40_generated = downsample_image(image_40_generated, image_40.shape)
    
    # Print dimensions for debugging
    print(f"{base_name}_20.png dimensions: {image_20.shape}")
    print(f"{base_name}_20_generated.png dimensions: {image_20_generated.shape}")
    print(f"{base_name}_40.png dimensions: {image_40.shape}")
    print(f"{base_name}_40_generated.png dimensions: {image_40_generated.shape}")
    
    if idx < 5:
        sample_images_20.append((image_20, image_20_generated))
        sample_images_40.append((image_40, image_40_generated))

    # Calculate metrics
    ssim_20 = ssim(image_20, image_20_generated, multichannel=True, win_size=3)
    psnr_20 = psnr(image_20, image_20_generated)
    mse_20 = mse(image_20, image_20_generated)
    fid_20, kid_20 = calculate_fid_kid(image_20, image_20_generated)
    
    ssim_40 = ssim(image_40, image_40_generated, multichannel=True, win_size=3)
    psnr_40 = psnr(image_40, image_40_generated)
    mse_40 = mse(image_40, image_40_generated)
    fid_40, kid_40 = calculate_fid_kid(image_40, image_40_generated)
    
    results.append({
        'file': base_name,
        'ssim_20': ssim_20, 'psnr_20': psnr_20, 'mse_20': mse_20, 'fid_20': fid_20, 'kid_20': kid_20,
        'ssim_40': ssim_40, 'psnr_40': psnr_40, 'mse_40': mse_40, 'fid_40': fid_40, 'kid_40': kid_40,
    })

# Plot sample images
fig, axs = plt.subplots(5, 4, figsize=(15, 15))
for i, ((orig_20, gen_20), (orig_40, gen_40)) in enumerate(zip(sample_images_20, sample_images_40)):
    axs[i, 0].imshow(cv2.cvtColor(orig_20, cv2.COLOR_BGR2RGB))
    axs[i, 0].set_title(f'Sample {i+1} - 20x Original')
    axs[i, 0].axis('off')
    
    axs[i, 1].imshow(cv2.cvtColor(gen_20, cv2.COLOR_BGR2RGB))
    axs[i, 1].set_title(f'Sample {i+1} - 20x Generated')
    axs[i, 1].axis('off')
    
    axs[i, 2].imshow(cv2.cvtColor(orig_40, cv2.COLOR_BGR2RGB))
    axs[i, 2].set_title(f'Sample {i+1} - 40x Original')
    axs[i, 2].axis('off')
    
    axs[i, 3].imshow(cv2.cvtColor(gen_40, cv2.COLOR_BGR2RGB))
    axs[i, 3].set_title(f'Sample {i+1} - 40x Generated')
    axs[i, 3].axis('off')

plt.tight_layout()
plt.show()

# Plot FID scores
fid_20_scores = [result['fid_20'] for result in results]
fid_40_scores = [result['fid_40'] for result in results]

x = np.arange(len(results))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, fid_20_scores, width, label='FID 20x')
rects2 = ax.bar(x + width/2, fid_40_scores, width, label='FID 40x')

ax.set_xlabel('Sample Index')
ax.set_ylabel('FID Score')
ax.set_title('FID Scores for 20x and 40x Images')
ax.legend()

plt.show()

# Save results to a CSV file
import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(output_path, 'comparison_metrics.csv'), index=False)

results_df.head()
 

100%|██████████| 75/75 [10:03<00:00,  8.04s/it]


217_20.png dimensions: (224, 224, 3)
217_20_generated.png dimensions: (224, 224, 3)
217_40.png dimensions: (224, 224, 3)
217_40_generated.png dimensions: (224, 224, 3)
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 31s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


ValueError: Non-matrix input to matrix function.